Kitti-dataset (conversion into COCO-Format)

In [2]:
import os
import json
import datetime
from PIL import Image

def kitti_to_coco(kitti_base_path, output_json_path):
    coco_format = {
        "info": {
            "description": "KITTI Dataset converted to COCO format",
            "url": "http://www.cvlibs.net/datasets/kitti/",
            "version": "1.0",
            "year": 2024,
            "contributor": "Your Name",
            "date_created": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        },
        "licenses": [
            {
                "id": 1,
                "name": "Attribution-NonCommercial-ShareAlike License",
                "url": "http://creativecommons.org/licenses/by-nc-sa/2.0/"
            }
        ],
        "images": [],
        "annotations": [],
        "categories": []
    }

    categories = ["Car", "Van", "Truck", "Pedestrian", "Person_sitting", "Cyclist", "Tram", "Misc"]
    for i, category in enumerate(categories):
        coco_format["categories"].append({
            "id": i + 1,
            "name": category,
            "supercategory": "none"
        })

    image_id = 1
    annotation_id = 1
    annotation_count = 0
    label_files_found = 0
    
    image_dir = os.path.join(kitti_base_path, "data_object_image_2", "training", "image_2")
    label_dir = os.path.join(kitti_base_path, "training", "label_2")
    
    print(f"Image directory: {image_dir}")
    print(f"Label directory: {label_dir}")
    
    for filename in os.listdir(image_dir):
        if filename.endswith(".png"):
            image_path = os.path.join(image_dir, filename)
            img = Image.open(image_path)
            width, height = img.size
            
            coco_format["images"].append({
                "id": image_id,
                "license": 1,
                "file_name": filename,
                "height": height,
                "width": width,
                "date_captured": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            })
            
            label_filename = os.path.splitext(filename)[0] + ".txt"
            label_path = os.path.join(label_dir, label_filename)
            
            if os.path.exists(label_path):
                label_files_found += 1
                with open(label_path, "r") as f:
                    for line in f:
                        parts = line.strip().split()
                        if len(parts) == 15:
                            category = parts[0]
                            if category in categories:
                                bbox = [float(parts[4]), float(parts[5]), float(parts[6]) - float(parts[4]), float(parts[7]) - float(parts[5])]
                                area = bbox[2] * bbox[3]
                                
                                coco_format["annotations"].append({
                                    "id": annotation_id,
                                    "image_id": image_id,
                                    "category_id": categories.index(category) + 1,
                                    "bbox": bbox,
                                    "area": area,
                                    "iscrowd": 0,
                                    "segmentation": []  # Empty segmentation as KITTI doesn't provide this
                                })
                                
                                annotation_id += 1
                                annotation_count += 1
            
            image_id += 1

    print(f"Processed {annotation_count} annotations")
    print(f"Found {label_files_found} label files")
    print(f"Total images: {len(coco_format['images'])}")
    print(f"Total annotations: {len(coco_format['annotations'])}")
    print(f"Total categories: {len(coco_format['categories'])}")

    with open(output_json_path, "w") as f:
        json.dump(coco_format, f, indent=2)

    # Verify the output
    with open(output_json_path, "r") as f:
        verified_coco = json.load(f)
        print("\nVerification:")
        print(f"Info section present: {'info' in verified_coco}")
        print(f"Licenses section present: {'licenses' in verified_coco}")
        print(f"Images section present: {'images' in verified_coco}")
        print(f"Annotations section present: {'annotations' in verified_coco}")
        print(f"Categories section present: {'categories' in verified_coco}")
        if verified_coco["images"]:
            print(f"Sample image entry: {verified_coco['images'][0]}")
        if verified_coco["annotations"]:
            print(f"Sample annotation entry: {verified_coco['annotations'][0]}")

if __name__ == "__main__":
    kitti_base_path = r"C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\kitti\kitti"
    output_json_path = r"C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\kitti\kitti_coco.json"
    kitti_to_coco(kitti_base_path, output_json_path)
    print("Conversion completed. COCO JSON file saved at:", output_json_path)


Image directory: C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\kitti\kitti\data_object_image_2\training\image_2
Label directory: C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\kitti\kitti\training\label_2
Processed 40570 annotations
Found 7481 label files
Total images: 7481
Total annotations: 40570
Total categories: 8

Verification:
Info section present: True
Licenses section present: True
Images section present: True
Annotations section present: True
Categories section present: True
Sample image entry: {'id': 1, 'license': 1, 'file_name': '000000.png', 'height': 370, 'width': 1224, 'date_captured': '2024-12-19 16:19:14'}
Sample annotation entry: {'id': 1, 'image_id': 1, 'category_id': 4, 'bbox': [712.4, 143.0, 98.33000000000004, 164.92000000000002], 'area': 16216.583600000009, 'iscrowd': 0, 'segmentation': []}
Conversion completed. COCO JSON file saved at: C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\k

Nuscence-dataset-Conversion COCO format

In [1]:
import json
import os
from datetime import datetime
from pyquaternion import Quaternion
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.geometry_utils import view_points
import numpy as np
from tqdm import tqdm

def validate_coco_format(coco_data):
    required_sections = ['info', 'licenses', 'images', 'annotations', 'categories']
    for section in required_sections:
        if section not in coco_data:
            print(f"Missing required section: {section}")
            return False
    
    # Additional validation checks
    if not coco_data['categories']:
        print("Categories section is empty")
        return False
    if not coco_data['annotations']:
        print("Annotations section is empty")
        return False
    return True

def nuscenes_to_coco(nuscenes_dir: str, output_dir: str):
    print("Loading NuScenes dataset...")
    nusc = NuScenes(version='v1.0-trainval', dataroot=nuscenes_dir, verbose=True)
    
    coco_format = {
        "info": {
            "description": "NuScenes Dataset in COCO format",
            "version": "1.0",
            "year": 2024,
            "contributor": "Converted from NuScenes",
            "date_created": datetime.now().isoformat()
        },
        "licenses": [
            {
                "id": 1,
                "name": "NuScenes License",
                "url": "https://www.nuscenes.org/terms-of-use"
            }
        ],
        "images": [],
        "annotations": [],
        "categories": []
    }

    print("Processing categories...")
    category_map = {}
    for idx, category in enumerate(nusc.category):
        category_id = idx + 1
        category_name = category['name']
        supercategory = category_name.split('.')[0]
        category_map[category_name] = category_id
        
        category_info = {
            'id': category_id,
            'name': category_name,
            'supercategory': supercategory
        }
        coco_format['categories'].append(category_info)

    print(f"Processed {len(category_map)} categories")

    print("Processing samples and annotations...")
    image_id = 1
    annotation_id = 1
    
    for sample in tqdm(nusc.sample, desc="Processing samples"):
        for camera_name in ['CAM_FRONT', 'CAM_FRONT_RIGHT', 'CAM_FRONT_LEFT', 'CAM_BACK', 'CAM_BACK_LEFT', 'CAM_BACK_RIGHT']:
            camera_token = sample['data'].get(camera_name)
            if not camera_token:
                continue
            
            camera_data = nusc.get('sample_data', camera_token)
            if camera_data['sensor_modality'] != 'camera':
                continue
            
            image_info = {
                'id': image_id,
                'file_name': os.path.basename(camera_data['filename']),
                'width': camera_data['width'],
                'height': camera_data['height'],
                'date_captured': datetime.fromtimestamp(sample['timestamp'] / 1000000).isoformat(),
                'license': 1,
                'coco_url': '',
                'flickr_url': ''
            }
            coco_format['images'].append(image_info)

            sensor = nusc.get('calibrated_sensor', camera_data['calibrated_sensor_token'])
            intrinsic = np.array(sensor['camera_intrinsic'])

            for ann_token in sample['anns']:
                ann = nusc.get('sample_annotation', ann_token)
                category_name = ann['category_name']
                
                if category_name not in category_map:
                    continue

                box = nusc.get_box(ann['token'])
                box.translate(-np.array(sensor['translation']))
                box.rotate(Quaternion(sensor['rotation']).inverse)
                
                corners_3d = box.corners()
                corners_2d = view_points(corners_3d, intrinsic, normalize=True)[:2, :]
                
                min_x, min_y = np.min(corners_2d, axis=1)
                max_x, max_y = np.max(corners_2d, axis=1)
                width = max_x - min_x
                height = max_y - min_y
                
                if width <= 0 or height <= 0:
                    continue

                segmentation = corners_2d.T.flatten().tolist()

                annotation = {
                    'id': annotation_id,
                    'image_id': image_id,
                    'category_id': category_map[category_name],
                    'bbox': [float(min_x), float(min_y), float(width), float(height)],
                    'area': float(width * height),
                    'segmentation': [segmentation],
                    'iscrowd': 0
                }
                coco_format['annotations'].append(annotation)
                annotation_id += 1
            
            image_id += 1

    if not validate_coco_format(coco_format):
        raise ValueError("Generated data does not meet COCO format requirements")

    print(f"Saving COCO format file to {output_dir}")
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    output_file = os.path.join(output_dir, 'nuscenes_coco.json')
    with open(output_file, 'w') as f:
        json.dump(coco_format, f)

    print(f"Conversion completed successfully!")
    print(f"Total images: {len(coco_format['images'])}")
    print(f"Total categories: {len(coco_format['categories'])}")
    print(f"Total annotations: {len(coco_format['annotations'])}")

if __name__ == "__main__":
    nuscenes_dir = r"C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\nuscene\nuscene\v1.0-trainval_meta"
    output_dir = r"C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\nuscene\output"
    nuscenes_to_coco(nuscenes_dir, output_dir)


Loading NuScenes dataset...
Loading NuScenes tables for version v1.0-trainval...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 82.935 seconds.
Reverse indexing ...
Done reverse indexing in 20.9 seconds.
Processing categories...
Processed 23 categories
Processing samples and annotations...


Processing samples: 100%|██████████| 34149/34149 [38:13<00:00, 14.89it/s]   


Saving COCO format file to C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\nuscene\output
Conversion completed successfully!
Total images: 204894
Total categories: 23
Total annotations: 6997122


Dataset Merging

In [ ]:
import json
from collections import defaultdict

def merge_coco_datasets(kitti_json_path, nuscenes_json_path, output_json_path):
    # Load KITTI and nuScenes COCO files
    with open(kitti_json_path, 'r') as f:
        kitti_data = json.load(f)
    with open(nuscenes_json_path, 'r') as f:
        nuscenes_data = json.load(f)

    # Initialize merged dataset
    merged_data = {
        "info": {
            "description": "Merged KITTI and nuScenes Dataset",
            "version": "1.0",
            "year": 2024,
            "contributor": "Your Name",
            "date_created": "2024-12-19"
        },
        "licenses": [],
        "images": [],
        "annotations": [],
        "categories": []
    }

    # Merge licenses (if different)
    license_ids = {}
    for dataset in [kitti_data, nuscenes_data]:
        for license in dataset.get('licenses', []):
            if license['name'] not in license_ids:
                new_id = len(merged_data['licenses']) + 1
                license_ids[license['name']] = new_id
                license['id'] = new_id
                merged_data['licenses'].append(license)

    # Merge categories
    category_ids = {}
    original_category_mapping = {}
    for dataset in [kitti_data, nuscenes_data]:
        for category in dataset['categories']:
            if category['name'] not in category_ids:
                new_id = len(merged_data['categories']) + 1
                category_ids[category['name']] = new_id
                original_category_mapping[category['id']] = new_id
                category['id'] = new_id
                merged_data['categories'].append(category)

    # Merge images and annotations
    image_id_mapping = {}
    annotation_id = 1

    for dataset in [kitti_data, nuscenes_data]:
        for image in dataset['images']:
            new_image_id = len(merged_data['images']) + 1
            image_id_mapping[image['id']] = new_image_id
            image['id'] = new_image_id
            merged_data['images'].append(image)

        for ann in dataset['annotations']:
            ann['id'] = annotation_id
            ann['image_id'] = image_id_mapping[ann['image_id']]
            
            # Handle category mapping
            if ann['category_id'] in original_category_mapping:
                ann['category_id'] = original_category_mapping[ann['category_id']]
            else:
                print(f"Warning: Category ID {ann['category_id']} not found in original categories. Skipping this annotation.")
                continue

            merged_data['annotations'].append(ann)
            annotation_id += 1

    # Save merged dataset
    with open(output_json_path, 'w') as f:
        json.dump(merged_data, f, indent=2)

    print(f"Merged dataset saved to {output_json_path}")
    print(f"Total images: {len(merged_data['images'])}")
    print(f"Total annotations: {len(merged_data['annotations'])}")
    print(f"Total categories: {len(merged_data['categories'])}")

# Usage
kitti_json_path = r"C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\kitti\kitti_coco.json"
nuscenes_json_path = r"C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\nuscene\output\nuscenes_coco.json"
output_json_path = r"C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\merged_dataset.json"

merge_coco_datasets(kitti_json_path, nuscenes_json_path, output_json_path)


Data validation 

This validation function will verify:
The presence of all required COCO format sections
The integrity of image and category references in annotations
The uniqueness of image and category IDs


If the validation passes, you can be confident that your merged dataset meets COCO format requirements. The function will also provide a summary of the total number of images, annotations, and categories in your merged dataset.


In [9]:
import json

def validate_coco_dataset(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    # Check for required fields
    required_fields = ['info', 'licenses', 'images', 'annotations', 'categories']
    for field in required_fields:
        if field not in data:
            print(f"Error: Missing required field '{field}'")
            return False
    
    # Validate images
    image_ids = set()
    for img in data['images']:
        if 'id' not in img or 'file_name' not in img:
            print("Error: Image missing required fields")
            return False
        image_ids.add(img['id'])
    
    # Validate categories
    category_ids = set()
    for cat in data['categories']:
        if 'id' not in cat or 'name' not in cat:
            print("Error: Category missing required fields")
            return False
        category_ids.add(cat['id'])
    
    # Validate annotations
    for ann in data['annotations']:
        if 'id' not in ann or 'image_id' not in ann or 'category_id' not in ann:
            print("Error: Annotation missing required fields")
            return False
        if ann['image_id'] not in image_ids:
            print(f"Error: Annotation refers to non-existent image ID {ann['image_id']}")
            return False
        if ann['category_id'] not in category_ids:
            print(f"Error: Annotation refers to non-existent category ID {ann['category_id']}")
            return False
    
    print("Validation successful!")
    print(f"Total images: {len(data['images'])}")
    print(f"Total annotations: {len(data['annotations'])}")
    print(f"Total categories: {len(data['categories'])}")
    return True

# Usage
file_path = r'C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\merged_dataset.json'
validate_coco_dataset(file_path)


Validation successful!
Total images: 212375
Total annotations: 7037692
Total categories: 31


True

Data Normalization and Standardization


In [ ]:
import os
import json
import shutil
from PIL import Image
import numpy as np

def copy_dataset_images(kitti_images_dir, nuscenes_images_dir, output_images_dir):
    os.makedirs(output_images_dir, exist_ok=True)
    print("Starting to copy images...")

    # Copy KITTI images
    kitti_training_dir = os.path.join(kitti_images_dir, "training", "image_2")
    print(f"Copying KITTI training images from: {kitti_training_dir}")
    if os.path.exists(kitti_training_dir):
        for file in os.listdir(kitti_training_dir):
            if file.endswith(('.png', '.jpg')):
                shutil.copy(os.path.join(kitti_training_dir, file), output_images_dir)
                print(f"Copied KITTI image: {file}")

    # Copy nuScenes images
    print(f"Copying nuScenes images from: {nuscenes_images_dir}")
    if os.path.exists(nuscenes_images_dir):
        for root, _, files in os.walk(nuscenes_images_dir):
            for file in files:
                if file.endswith(('.png', '.jpg')):
                    shutil.copy(os.path.join(root, file), output_images_dir)
                    print(f"Copied nuScenes image: {file}")

def normalize_and_standardize_dataset(merged_json_path, images_dir, output_json_path, target_size=(640, 640)):
    try:
        # Load merged COCO dataset
        print(f"Loading merged dataset from: {merged_json_path}")
        with open(merged_json_path, 'r') as f:
            data = json.load(f)
        print(f"Successfully loaded JSON file with {len(data['images'])} images")

        # Create output directory for resized images
        output_images_dir = os.path.join(images_dir, "normalized_images")
        os.makedirs(output_images_dir, exist_ok=True)
        print(f"Created output directory at {output_images_dir}")

        processed_count = 0
        for image in data['images']:
            try:
                image_id = image['id']
                file_name = image['file_name']
                
                # Load the original image
                image_path = os.path.join(images_dir, file_name)
                if not os.path.exists(image_path):
                    print(f"Warning: Image not found: {image_path}")
                    continue
                
                img = Image.open(image_path)
                
                # Resize image
                img_resized = img.resize(target_size, Image.Resampling.LANCZOS)
                
                # Save resized image
                resized_file_name = f"resized_{file_name}"
                img_resized.save(os.path.join(output_images_dir, resized_file_name))

                # Update image dimensions in data
                width, height = img_resized.size
                image['width'] = width
                image['height'] = height
                image['file_name'] = resized_file_name

                # Normalize annotations for this image
                for ann in data['annotations']:
                    if ann['image_id'] == image_id:
                        x_min, y_min, bbox_width, bbox_height = ann['bbox']
                        normalized_bbox = [
                            x_min / width,
                            y_min / height,
                            bbox_width / width,
                            bbox_height / height
                        ]
                        ann['bbox'] = normalized_bbox

                processed_count += 1
                if processed_count % 100 == 0:
                    print(f"Processed {processed_count} images")

            except Exception as e:
                print(f"Error processing image {file_name}: {str(e)}")

        # Save the updated dataset
        with open(output_json_path, 'w') as f:
            json.dump(data, f, indent=2)

        print(f"Normalization completed. Processed {processed_count} images")
        print(f"Updated dataset saved to {output_json_path}")
        print(f"Normalized images saved to {output_images_dir}")
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")

# Correct paths based on your directory structure
kitti_images_dir = r"C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\kitti\kitti\data_object_image_2"
nuscenes_images_dir = r"C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\nuscene\nuscene\v1.0-trainval\samples"
output_dir = r"C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\images"
merged_json_path = r"C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\merged_dataset.json"
output_json_path = r"C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\normalized_dataset.json"

# First copy all images to a single directory
copy_dataset_images(kitti_images_dir, nuscenes_images_dir, output_dir)

# Then normalize the dataset
normalize_and_standardize_dataset(merged_json_path, output_dir, output_json_path)


Starting to copy images...
Copying KITTI training images from: C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\kitti\kitti\data_object_image_2\training\image_2
Copied KITTI image: 000000.png
Copied KITTI image: 000001.png
Copied KITTI image: 000002.png
Copied KITTI image: 000003.png
Copied KITTI image: 000004.png
Copied KITTI image: 000005.png
Copied KITTI image: 000006.png
Copied KITTI image: 000007.png
Copied KITTI image: 000008.png
Copied KITTI image: 000009.png
Copied KITTI image: 000010.png
Copied KITTI image: 000011.png
Copied KITTI image: 000012.png
Copied KITTI image: 000013.png
Copied KITTI image: 000014.png
Copied KITTI image: 000015.png
Copied KITTI image: 000016.png
Copied KITTI image: 000017.png
Copied KITTI image: 000018.png
Copied KITTI image: 000019.png
Copied KITTI image: 000020.png
Copied KITTI image: 000021.png
Copied KITTI image: 000022.png
Copied KITTI image: 000023.png
Copied KITTI image: 000024.png
Copied KITTI image: 000025.png
Copied K

In [17]:
import os
import json
import shutil
from PIL import Image
import numpy as np
from tqdm import tqdm

def copy_dataset_images(kitti_images_dir, nuscenes_images_dir, output_images_dir):
    os.makedirs(output_images_dir, exist_ok=True)
    print("Starting to copy images...")

    # Copy KITTI images
    kitti_training_dir = os.path.join(kitti_images_dir, "training", "image_2")
    if os.path.exists(kitti_training_dir):
        files = [f for f in os.listdir(kitti_training_dir) if f.endswith(('.png', '.jpg'))]
        for file in tqdm(files, desc="Copying KITTI images"):
            shutil.copy(os.path.join(kitti_training_dir, file), output_images_dir)

    # Copy nuScenes images
    if os.path.exists(nuscenes_images_dir):
        all_files = []
        for root, _, files in os.walk(nuscenes_images_dir):
            for file in files:
                if file.endswith(('.png', '.jpg')):
                    all_files.append((root, file))
        
        for root, file in tqdm(all_files, desc="Copying nuScenes images"):
            shutil.copy(os.path.join(root, file), output_images_dir)

def normalize_and_standardize_dataset(merged_json_path, images_dir, output_json_path, target_size=(640, 640)):
    try:
        # Load merged COCO dataset
        print("Loading merged dataset...")
        with open(merged_json_path, 'r') as f:
            data = json.load(f)
        print(f"Loaded {len(data['images'])} images from JSON")

        # Create output directory
        output_images_dir = os.path.join(images_dir, "normalized_images")
        os.makedirs(output_images_dir, exist_ok=True)

        # Pre-process annotations to group them by image_id
        print("Organizing annotations...")
        annotations_by_image = {}
        for ann in data['annotations']:
            img_id = ann['image_id']
            if img_id not in annotations_by_image:
                annotations_by_image[img_id] = []
            annotations_by_image[img_id].append(ann)

        # Process images with progress bar
        processed_images = []
        processed_annotations = []
        skipped_images = 0

        print("Processing images and normalizing annotations...")
        for image in tqdm(data['images'], desc="Processing"):
            try:
                image_id = image['id']
                file_name = image['file_name']
                
                # Load original image
                image_path = os.path.join(images_dir, file_name)
                if not os.path.exists(image_path):
                    skipped_images += 1
                    continue
                
                # Process image
                with Image.open(image_path) as img:
                    original_width, original_height = img.size
                    img_resized = img.resize(target_size, Image.Resampling.LANCZOS)
                    
                    # Save resized image
                    resized_file_name = f"resized_{file_name}"
                    img_resized.save(os.path.join(output_images_dir, resized_file_name))

                # Update image info
                image['file_name'] = resized_file_name
                image['width'] = target_size[0]
                image['height'] = target_size[1]
                processed_images.append(image)

                # Normalize annotations for this image
                if image_id in annotations_by_image:
                    for ann in annotations_by_image[image_id]:
                        x_min, y_min, bbox_width, bbox_height = ann['bbox']
                        
                        # First normalize to original image dimensions
                        normalized_bbox = [
                            x_min / original_width,
                            y_min / original_height,
                            bbox_width / original_width,
                            bbox_height / original_height
                        ]
                        
                        # Ensure values are in [0, 1] range
                        normalized_bbox = [
                            min(max(0.0, coord), 1.0) for coord in normalized_bbox
                        ]
                        
                        ann['bbox'] = normalized_bbox
                        processed_annotations.append(ann)

            except Exception as e:
                print(f"Error processing {file_name}: {str(e)}")
                skipped_images += 1
                continue

        # Update dataset with processed data
        data['images'] = processed_images
        data['annotations'] = processed_annotations

        # Save normalized dataset
        print("Saving normalized dataset...")
        with open(output_json_path, 'w') as f:
            json.dump(data, f, indent=2)

        print(f"\nNormalization Summary:")
        print(f"Total images processed: {len(processed_images)}")
        print(f"Total annotations processed: {len(processed_annotations)}")
        print(f"Skipped images: {skipped_images}")
        print(f"Output saved to: {output_json_path}")
        
    except Exception as e:
        print(f"Fatal error: {str(e)}")
        raise

# Paths configuration
kitti_images_dir = r"C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\kitti\kitti\data_object_image_2"
nuscenes_images_dir = r"C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\nuscene\nuscene\v1.0-trainval\samples"
output_dir = r"C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\images"
merged_json_path = r"C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\merged_dataset.json"
output_json_path = r"C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\normalized_dataset.json"

# Execute the pipeline
if __name__ == "__main__":
    print("Starting dataset normalization pipeline...")
    copy_dataset_images(kitti_images_dir, nuscenes_images_dir, output_dir)
    normalize_and_standardize_dataset(merged_json_path, output_dir, output_json_path)


Starting dataset normalization pipeline...
Starting to copy images...


Copying KITTI images: 100%|██████████| 7481/7481 [00:52<00:00, 141.48it/s]


Loading merged dataset...
Loaded 212375 images from JSON
Organizing annotations...
Processing images and normalizing annotations...


Processing: 100%|██████████| 212375/212375 [25:12<00:00, 140.43it/s]  


Saving normalized dataset...

Normalization Summary:
Total images processed: 7481
Total annotations processed: 40570
Skipped images: 204894
Output saved to: C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\normalized_dataset.json


In [ ]:
import os
import json
import shutil
from PIL import Image
import numpy as np
from tqdm import tqdm

# Define paths
kitti_images_dir = r"C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\kitti\kitti\data_object_image_2"
nuscenes_base_dir = r"C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\nuscene\nuscene"
output_dir = r"C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\images"
merged_json_path = r"C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\merged_dataset.json"
output_json_path = r"C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\normalized_dataset.json"

def copy_dataset_images(kitti_images_dir, nuscenes_base_dir, output_images_dir):
    os.makedirs(output_images_dir, exist_ok=True)
    print("Starting to copy images...")
    total_copied = 0

    # Copy KITTI images
    kitti_training_dir = os.path.join(kitti_images_dir, "training", "image_2")
    print(f"Processing KITTI images from: {kitti_training_dir}")
    if os.path.exists(kitti_training_dir):
        files = [f for f in os.listdir(kitti_training_dir) if f.endswith(('.png', '.jpg'))]
        for file in tqdm(files, desc="Copying KITTI images"):
            dest_file = f"kitti_{file}"
            shutil.copy(os.path.join(kitti_training_dir, file), 
                       os.path.join(output_images_dir, dest_file))
            total_copied += 1

    # Process nuScenes metadata
    sample_data_path = os.path.join(nuscenes_base_dir, "v1.0-trainval_meta", "v1.0-trainval", "sample_data.json")
    print(f"\nLoading nuScenes metadata from: {sample_data_path}")
    
    if os.path.exists(sample_data_path):
        with open(sample_data_path, 'r') as f:
            sample_data = json.load(f)
        print(f"Loaded {len(sample_data)} sample data entries")
        
        print("Processing nuScenes images...")
        for data in tqdm(sample_data, desc="Processing nuScenes entries"):
            if data.get('sensor_modality') == 'camera':
                filename = data.get('filename', '')
                if filename:
                    source_path = os.path.join(nuscenes_base_dir, "v1.0-trainval", filename)
                    if os.path.exists(source_path):
                        camera_type = data.get('channel', 'CAM')
                        dest_file = f"nuscenes_{camera_type}_{os.path.basename(filename)}"
                        shutil.copy(source_path, os.path.join(output_images_dir, dest_file))
                        total_copied += 1

    print(f"\nTotal images copied: {total_copied}")
    return total_copied

def normalize_and_standardize_dataset(merged_json_path, images_dir, output_json_path, target_size=(640, 640)):
    try:
        output_images_dir = os.path.join(images_dir, "normalized_images")
        os.makedirs(output_images_dir, exist_ok=True)
        print("Created output directory for normalized images")

        print("Loading merged dataset...")
        try:
            with open(merged_json_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            print(f"Successfully loaded dataset with {len(data['images'])} images")
        except json.JSONDecodeError as e:
            print(f"Error reading JSON file: {str(e)}")
            return

        processed_images = []
        processed_annotations = []
        skipped_images = 0
        
        # Create lookup table for annotations
        annotations_by_image = {}
        for ann in data['annotations']:
            img_id = ann['image_id']
            if img_id not in annotations_by_image:
                annotations_by_image[img_id] = []
            annotations_by_image[img_id].append(ann)

        # Process images in smaller batches
        batch_size = 50
        for i in range(0, len(data['images']), batch_size):
            batch = data['images'][i:min(i + batch_size, len(data['images']))]
            
            for image in tqdm(batch, desc=f"Processing batch {i//batch_size + 1}"):
                try:
                    image_path = os.path.join(images_dir, image['file_name'])
                    if not os.path.exists(image_path):
                        skipped_images += 1
                        continue

                    with Image.open(image_path) as img:
                        original_width, original_height = img.size
                        img_resized = img.resize(target_size, Image.Resampling.LANCZOS)
                        resized_file_name = f"resized_{image['file_name']}"
                        img_resized.save(os.path.join(output_images_dir, resized_file_name))

                    image['file_name'] = resized_file_name
                    image['width'] = target_size[0]
                    image['height'] = target_size[1]
                    processed_images.append(image)

                    if image['id'] in annotations_by_image:
                        for ann in annotations_by_image[image['id']]:
                            bbox = ann['bbox']
                            normalized_bbox = [
                                bbox[0] / original_width,
                                bbox[1] / original_height,
                                bbox[2] / original_width,
                                bbox[3] / original_height
                            ]
                            ann['bbox'] = [max(0, min(1, coord)) for coord in normalized_bbox]
                            processed_annotations.append(ann)

                except Exception as e:
                    print(f"Error processing image {image['file_name']}: {str(e)}")
                    skipped_images += 1

        normalized_data = {
            "info": data.get("info", {}),
            "licenses": data.get("licenses", []),
            "categories": data.get("categories", []),
            "images": processed_images,
            "annotations": processed_annotations
        }

        print("Saving normalized dataset...")
        with open(output_json_path, 'w') as f:
            json.dump(normalized_data, f)

        print(f"\nNormalization Summary:")
        print(f"Total images processed: {len(processed_images)}")
        print(f"Total annotations processed: {len(processed_annotations)}")
        print(f"Skipped images: {skipped_images}")
        
    except Exception as e:
        print(f"Fatal error: {str(e)}")
        raise

if __name__ == "__main__":
    try:
        print("Starting dataset normalization...")
        total_images = copy_dataset_images(kitti_images_dir, nuscenes_base_dir, output_dir)
        if total_images > 0:
            print("\nProceeding with normalization...")
            normalize_and_standardize_dataset(merged_json_path, output_dir, output_json_path)
        else:
            print("No images were copied. Please check the source directories.")
    except Exception as e:
        print(f"Error during execution: {str(e)}")


Starting dataset normalization...
Starting to copy images...
Processing KITTI images from: C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\kitti\kitti\data_object_image_2\training\image_2


Copying KITTI images: 100%|██████████| 7481/7481 [02:10<00:00, 57.43it/s]



Loading nuScenes metadata from: C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\nuscene\nuscene\v1.0-trainval_meta\v1.0-trainval\sample_data.json
Loaded 2631083 sample data entries
Processing nuScenes images...


Processing nuScenes entries: 100%|██████████| 2631083/2631083 [00:02<00:00, 883578.11it/s] 



Total images copied: 7481

Proceeding with normalization...
Created output directory for normalized images
Loading merged dataset...


In [ ]:
import os
import json
import shutil
from PIL import Image
import numpy as np
from tqdm import tqdm

# Define paths
kitti_images_dir = r"C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\kitti\kitti\data_object_image_2"
nuscenes_base_dir = r"C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\nuscene\nuscene"
output_dir = r"C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\images"
merged_json_path = r"C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\merged_dataset.json"
output_json_path = r"C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\normalized_dataset.json"

def process_json_file(file_path, batch_size=1000):
    print(f"Processing JSON file: {os.path.getsize(file_path)/1024/1024:.2f} MB")
    with open(file_path, 'r') as f:
        # Read header information
        header = ""
        while True:
            char = f.read(1)
            header += char
            if '"images":' in header:
                break
        
        # Process images array
        images = []
        current_chunk = ""
        for line in f:
            if ']' in line:  # End of images array
                break
            current_chunk += line
            if len(current_chunk) >= batch_size:
                try:
                    
                    chunk_data = json.loads("[" + current_chunk.rstrip(',\n') + "]")
                    images.extend(chunk_data)
                    current_chunk = ""
                except json.JSONDecodeError:
                    continue
    return images

def copy_dataset_images(kitti_images_dir, nuscenes_base_dir, output_images_dir):
    os.makedirs(output_images_dir, exist_ok=True)
    print("Starting to copy images...")
    total_copied = 0

    # Copy KITTI images
    kitti_training_dir = os.path.join(kitti_images_dir, "training", "image_2")
    print(f"Processing KITTI images from: {kitti_training_dir}")
    if os.path.exists(kitti_training_dir):
        files = [f for f in os.listdir(kitti_training_dir) if f.endswith(('.png', '.jpg'))]
        for file in tqdm(files, desc="Copying KITTI images"):
            dest_file = f"kitti_{file}"
            if not os.path.exists(os.path.join(output_images_dir, dest_file)):
                shutil.copy(os.path.join(kitti_training_dir, file), 
                          os.path.join(output_images_dir, dest_file))
            total_copied += 1

    # Process nuScenes images
    nuscenes_samples_dir = os.path.join(nuscenes_base_dir, "v1.0-trainval", "samples")
    camera_dirs = ['CAM_FRONT', 'CAM_FRONT_RIGHT', 'CAM_FRONT_LEFT', 
                  'CAM_BACK', 'CAM_BACK_LEFT', 'CAM_BACK_RIGHT']
    
    for camera in camera_dirs:
        camera_path = os.path.join(nuscenes_samples_dir, camera)
        print(f"\nProcessing {camera} images from: {camera_path}")
        
        if os.path.exists(camera_path):
            files = [f for f in os.listdir(camera_path) if f.endswith(('.png', '.jpg'))]
            for file in tqdm(files, desc=f"Copying {camera} images"):
                dest_file = f"nuscenes_{camera}_{file}"
                if not os.path.exists(os.path.join(output_images_dir, dest_file)):
                    shutil.copy(os.path.join(camera_path, file), 
                              os.path.join(output_images_dir, dest_file))
                total_copied += 1

    print(f"\nTotal images copied: {total_copied}")
    return total_copied

def normalize_and_standardize_dataset(merged_json_path, images_dir, output_json_path, target_size=(640, 640)):
    try:
        output_images_dir = os.path.join(images_dir, "normalized_images")
        os.makedirs(output_images_dir, exist_ok=True)
        print("Created output directory for normalized images")

        print("Loading merged dataset...")
        images = process_json_file(merged_json_path)
        print(f"Processing {len(images)} images")

        processed_images = []
        processed_annotations = []
        skipped_images = 0
        batch_size = 50

        for i in range(0, len(images), batch_size):
            batch = images[i:i + batch_size]
            print(f"\nProcessing batch {i//batch_size + 1}/{len(images)//batch_size + 1}")
            
            for image in tqdm(batch):
                try:
                    image_path = os.path.join(images_dir, image['file_name'])
                    if not os.path.exists(image_path):
                        skipped_images += 1
                        continue

                    with Image.open(image_path) as img:
                        original_width, original_height = img.size
                        img_resized = img.resize(target_size, Image.Resampling.LANCZOS)
                        resized_file_name = f"resized_{image['file_name']}"
                        img_resized.save(os.path.join(output_images_dir, resized_file_name))

                    processed_images.append({
                        'id': image['id'],
                        'file_name': resized_file_name,
                        'width': target_size[0],
                        'height': target_size[1]
                    })

                except Exception as e:
                    print(f"Error processing image {image.get('file_name', 'unknown')}: {str(e)}")
                    skipped_images += 1

            if (i + batch_size) % (batch_size * 10) == 0:
                print(f"Progress: {len(processed_images)}/{len(images)} images processed")

        print("Saving normalized dataset...")
        normalized_data = {
            "images": processed_images,
            "annotations": processed_annotations
        }

        with open(output_json_path, 'w') as f:
            json.dump(normalized_data, f)

        print(f"\nNormalization Summary:")
        print(f"Total images processed: {len(processed_images)}")
        print(f"Total annotations processed: {len(processed_annotations)}")
        print(f"Skipped images: {skipped_images}")
        
    except Exception as e:
        print(f"Fatal error: {str(e)}")
        raise

if __name__ == "__main__":
    try:
        print("Starting dataset normalization...")
        total_images = copy_dataset_images(kitti_images_dir, nuscenes_base_dir, output_dir)
        if total_images > 0:
            print("\nProceeding with normalization...")
            normalize_and_standardize_dataset(merged_json_path, output_dir, output_json_path)
        else:
            print("No images were copied. Please check the source directories.")
    except Exception as e:
        print(f"Error during execution: {str(e)}")


Starting dataset normalization...
Starting to copy images...
Processing KITTI images from: C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\kitti\kitti\data_object_image_2\training\image_2


Copying KITTI images: 100%|██████████| 7481/7481 [00:00<00:00, 7785.40it/s]



Processing CAM_FRONT images from: C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\nuscene\nuscene\v1.0-trainval\samples\CAM_FRONT

Processing CAM_FRONT_RIGHT images from: C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\nuscene\nuscene\v1.0-trainval\samples\CAM_FRONT_RIGHT

Processing CAM_FRONT_LEFT images from: C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\nuscene\nuscene\v1.0-trainval\samples\CAM_FRONT_LEFT

Processing CAM_BACK images from: C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\nuscene\nuscene\v1.0-trainval\samples\CAM_BACK

Processing CAM_BACK_LEFT images from: C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\nuscene\nuscene\v1.0-trainval\samples\CAM_BACK_LEFT

Processing CAM_BACK_RIGHT images from: C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\nuscene\nuscene\v1.0-trainval\samples\CAM_BACK_RIGHT

Total images copied

In [ ]:
import os
import json
import shutil
from PIL import Image
import numpy as np
from tqdm import tqdm
import gc
import time
import psutil

# Define paths
kitti_images_dir = r"C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\kitti\kitti\data_object_image_2"
nuscenes_base_dir = r"C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\nuscene\nuscene"
output_dir = r"C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\images"
merged_json_path = r"C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\merged_dataset.json"
output_json_path = r"C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\normalized_dataset.json"

def copy_dataset_images(kitti_images_dir, nuscenes_base_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    print("Starting to copy images...")
    total_copied = 0

    # Copy KITTI images
    kitti_training_dir = os.path.join(kitti_images_dir, "training", "image_2")
    print(f"Processing KITTI images from: {kitti_training_dir}")
    if os.path.exists(kitti_training_dir):
        files = [f for f in os.listdir(kitti_training_dir) if f.endswith(('.png', '.jpg'))]
        for file in tqdm(files, desc="Copying KITTI images"):
            dest_file = f"kitti_{file}"
            shutil.copy(os.path.join(kitti_training_dir, file), 
                       os.path.join(output_dir, dest_file))
            total_copied += 1

def copy_dataset_images(kitti_images_dir, nuscenes_base_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    print("Starting to copy images...")
    total_copied = 0

    # Copy KITTI images
    kitti_training_dir = os.path.join(kitti_images_dir, "training", "image_2")
    print(f"Processing KITTI images from: {kitti_training_dir}")
    if os.path.exists(kitti_training_dir):
        files = [f for f in os.listdir(kitti_training_dir) if f.endswith(('.png', '.jpg'))]
        for file in tqdm(files, desc="Copying KITTI images"):
            dest_file = f"kitti_{file}"
            shutil.copy(os.path.join(kitti_training_dir, file), 
                       os.path.join(output_dir, dest_file))
            total_copied += 1

    # Define nuScenes specific paths
    nuscenes_meta_dir = os.path.join(nuscenes_base_dir, "v1.0-trainval_meta", "v1.0-trainval")
    nuscenes_samples_dir = os.path.join(nuscenes_base_dir, "samples")  # Changed from v1.0-trainval
    sample_data_path = os.path.join(nuscenes_meta_dir, "sample_data.json")

    # Process nuScenes images
    if os.path.exists(sample_data_path):
        with open(sample_data_path, 'r') as f:
            sample_data = json.load(f)
        print(f"Loaded {len(sample_data)} sample data entries")
        
        print("Processing nuScenes images...")
        for data in tqdm(sample_data, desc="Processing nuScenes entries"):
            if data.get('sensor_modality') == 'camera':
                filename = data.get('filename', '')
                if filename:
                    # Modified path construction
                    source_path = os.path.join(nuscenes_samples_dir, filename)
                    if os.path.exists(source_path):
                        camera_type = data.get('channel', 'CAM')
                        dest_file = f"nuscenes_{camera_type}_{os.path.basename(filename)}"
                        shutil.copy(source_path, os.path.join(output_dir, dest_file))
                        total_copied += 1

    print(f"\nTotal images copied: {total_copied}")
    return total_copied


    print(f"\nTotal images copied: {total_copied}")
    return total_copied

def process_json_in_chunks(file_path, chunk_size=1024*1024):
    print(f"Processing JSON file ({os.path.getsize(file_path)/1024/1024:.2f} MB)")
    with open(file_path, 'r') as f:
        content = ""
        total_size = os.path.getsize(file_path)
        processed_size = 0
        
        with tqdm(total=total_size, desc="Loading JSON") as pbar:
            while True:
                chunk = f.read(chunk_size)
                if not chunk:
                    break
                content += chunk
                processed_size += len(chunk)
                pbar.update(len(chunk))
                
                if processed_size % (100*1024*1024) == 0:  # Status every 100MB
                    print(f"Memory usage: {psutil.Process().memory_info().rss / 1024 / 1024:.2f} MB")
                    gc.collect()
    
    return json.loads(content)

def normalize_and_standardize_dataset(merged_json_path, output_dir, output_json_path, target_size=(640, 640)):
    try:
        output_images_dir = os.path.join(output_dir, "normalized_images")
        os.makedirs(output_images_dir, exist_ok=True)
        print("Created output directory for normalized images")

        # Load JSON data with progress tracking
        data = process_json_in_chunks(merged_json_path)
        total_images = len(data['images'])
        print(f"Processing {total_images} images")

        processed_images = []
        processed_annotations = []
        skipped_images = 0
        batch_size = 50

        # Process images in batches
        for i in range(0, total_images, batch_size):
            batch = data['images'][i:min(i + batch_size, total_images)]
            print(f"\nProcessing batch {i//batch_size + 1}/{(total_images-1)//batch_size + 1}")
            
            for image in tqdm(batch, desc=f"Batch {i//batch_size + 1}"):
                try:
                    image_path = os.path.join(output_dir, image['file_name'])
                    if not os.path.exists(image_path):
                        print(f"Skipping missing image: {image['file_name']}")
                        skipped_images += 1
                        continue

                    with Image.open(image_path) as img:
                        original_width, original_height = img.size
                        img_resized = img.resize(target_size, Image.Resampling.LANCZOS)
                        resized_file_name = f"resized_{image['file_name']}"
                        img_resized.save(os.path.join(output_images_dir, resized_file_name))

                    image['file_name'] = resized_file_name
                    image['width'] = target_size[0]
                    image['height'] = target_size[1]
                    processed_images.append(image)

                    # Process annotations
                    image_annotations = [ann for ann in data['annotations'] 
                                      if ann['image_id'] == image['id']]
                    for ann in image_annotations:
                        bbox = ann['bbox']
                        normalized_bbox = [
                            bbox[0] / original_width,
                            bbox[1] / original_height,
                            bbox[2] / original_width,
                            bbox[3] / original_height
                        ]
                        ann['bbox'] = [max(0, min(1, coord)) for coord in normalized_bbox]
                        processed_annotations.append(ann)

                except Exception as e:
                    print(f"Error processing {image['file_name']}: {str(e)}")
                    skipped_images += 1

            # Garbage collection after each batch
            gc.collect()

        # Save final results
        normalized_data = {
            "info": data.get("info", {}),
            "licenses": data.get("licenses", []),
            "categories": data.get("categories", []),
            "images": processed_images,
            "annotations": processed_annotations
        }

        print("\nSaving normalized dataset...")
        with open(output_json_path, 'w') as f:
            json.dump(normalized_data, f)

        print(f"\nNormalization Summary:")
        print(f"Total images processed: {len(processed_images)}")
        print(f"Total annotations processed: {len(processed_annotations)}")
        print(f"Skipped images: {skipped_images}")

    except Exception as e:
        print(f"Fatal error: {str(e)}")
        raise

if __name__ == "__main__":
    try:
        print("Starting dataset normalization...")
        total_images = copy_dataset_images(kitti_images_dir, nuscenes_base_dir, output_dir)
        if total_images > 0:
            print("\nProceeding with normalization...")
            normalize_and_standardize_dataset(merged_json_path, output_dir, output_json_path)
        else:
            print("No images were copied. Please check the source directories.")
    except Exception as e:
        print(f"Error during execution: {str(e)}")


Starting dataset normalization...
Starting to copy images...
Processing KITTI images from: C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\kitti\kitti\data_object_image_2\training\image_2


Copying KITTI images: 100%|██████████| 7481/7481 [00:33<00:00, 223.37it/s]


Loaded 2631083 sample data entries
Processing nuScenes images...


Processing nuScenes entries: 100%|██████████| 2631083/2631083 [00:03<00:00, 680115.86it/s] 



Total images copied: 7481

Proceeding with normalization...
Created output directory for normalized images
Processing JSON file (5593.78 MB)


...

Loading JSON:  20%|█▉        | 1153433600/5865501796 [06:23<1:00:18, 1302207.67it/s]

Memory usage: 1109.59 MB


Loading JSON:  21%|██▏       | 1259339776/5865501796 [07:43<1:10:21, 1091112.06it/s]

Memory usage: 1211.68 MB


Loading JSON:  23%|██▎       | 1339031552/5865501796 [08:53<1:46:15, 709973.36it/s] 

Normalization Dataset Verification:

In [ ]:
import os
import json
from PIL import Image

def verify_dataset_normalization(normalized_images_dir, normalized_json_path):
    print("Starting dataset verification...")
    
    # 1. Verify images
    print("\nChecking image dimensions...")
    if os.path.exists(normalized_images_dir):
        incorrect_images = []
        total_images = 0
        
        for file_name in os.listdir(normalized_images_dir):
            if file_name.endswith(('.png', '.jpg')):
                total_images += 1
                image_path = os.path.join(normalized_images_dir, file_name)
                with Image.open(image_path) as img:
                    if img.size != (640, 640):
                        incorrect_images.append(f"{file_name}: {img.size}")
                        
                if total_images % 1000 == 0:
                    print(f"Processed {total_images} images...")
        
        print(f"\nImage Verification Results:")
        print(f"Total images processed: {total_images}")
        print(f"Images with incorrect dimensions: {len(incorrect_images)}")
        if incorrect_images:
            print("First 5 incorrect images:")
            for img in incorrect_images[:5]:
                print(img)
    else:
        print(f"Error: Normalized images directory not found at {normalized_images_dir}")
    
    # 2. Verify JSON file
    print("\nChecking normalized_dataset.json...")
    if os.path.exists(normalized_json_path):
        with open(normalized_json_path, 'r') as f:
            try:
                data = json.load(f)
                
                # Check image dimensions in JSON
                incorrect_dimensions = []
                for img in data['images']:
                    if img['width'] != 640 or img['height'] != 640:
                        incorrect_dimensions.append(f"Image ID {img['id']}: {img['width']}x{img['height']}")
                
                # Check bbox normalization
                incorrect_bbox = []
                for ann in data['annotations']:
                    bbox = ann['bbox']
                    if not all(0 <= coord <= 1 for coord in bbox):
                        incorrect_bbox.append(f"Annotation ID {ann['id']}: {bbox}")
                
                print("\nJSON Verification Results:")
                print(f"Total images in JSON: {len(data['images'])}")
                print(f"Total annotations: {len(data['annotations'])}")
                print(f"Images with incorrect dimensions: {len(incorrect_dimensions)}")
                print(f"Annotations with non-normalized bbox: {len(incorrect_bbox)}")
                
                if incorrect_dimensions:
                    print("\nFirst 5 incorrect dimensions:")
                    for dim in incorrect_dimensions[:5]:
                        print(dim)
                
                if incorrect_bbox:
                    print("\nFirst 5 non-normalized bounding boxes:")
                    for bbox in incorrect_bbox[:5]:
                        print(bbox)
                        
            except json.JSONDecodeError:
                print("Error: Invalid JSON file format")
    else:
        print(f"Error: Normalized dataset JSON not found at {normalized_json_path}")

# Usage
normalized_images_dir = r"C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\images\normalized_images"
normalized_json_path = r"C:\Users\chand\Documents\Object-Detection-Autonomus Vehicles - CV Project\normalized_dataset.json"

verify_dataset_normalization(normalized_images_dir, normalized_json_path)


Starting dataset verification...

Checking image dimensions...
Processed 1000 images...
Processed 2000 images...
Processed 3000 images...
Processed 4000 images...
Processed 5000 images...
